# Install and import libraries

In [1]:
# !pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
# !pip install -qq accelerate==0.26.1
# !apt install git-lfs

In [2]:
!pip install -qq datasets evaluate transformers[sentencepiece]
!pip install -qq accelerate
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.

In [3]:
import os
import math
import torch
import pandas as pd
import re

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from huggingface_hub import notebook_login

In [36]:
notebook_login()

# Read dataset

In [5]:
DATASET_PATH = 'that_ngon_tu_tuyet_final.csv'
df = pd.read_csv(DATASET_PATH)
df

,content,url
0,"Giàu làm chị, khó làm em, Sang chớ kiêu căng, ...",https://www.thivien.net/Nguy%E1%BB%85n-B%E1%BB...
1,"Nhất bộc rành rành lại thập thành, Ở cho thực ...",https://www.thivien.net/Tr%E1%BB%8Bnh-Doanh/Ba...
2,"Văn chương phú lục đã xong rồi, Thừa giấy làm ...",https://www.thivien.net/Nguy%E1%BB%85n-Qu%E1%B...
3,"Đồn Phố Hiến vui hơn kinh kỳ, Chơi ba ngày chẳ...",https://www.thivien.net/Nguy%E1%BB%85n-Qu%E1%B...
4,"Quan nhậm châu khách Hoan châu, Cầm bằng mà cũ...",https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi...
...,...,...
285,Từng đêm xuân cháy từng biển mộng Mênh mông ru...,https://www.thivien.net/H%C3%A0n-Qu%E1%BB%91c-...
286,Người ngồi như cỏ hoang vu quá! Phố chết tưng ...,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...
287,Chỉ giống người trên mặt địa cầu Đang tâm hành...,https://www.thivien.net/V%C5%A9-Ho%C3%A0ng-Ch%...
288,Hồ đã tan thây sóng Bạch Đằng Minh còn mất vía...,https://www.thivien.net/V%C5%A9-Ho%C3%A0ng-Ch%...


# Prepare dataset

In [6]:
def split_content(content):

        content = content.strip()

        raw_tokens = re.split(r'\s+', content)

        tokens = []
        for token in raw_tokens:
            if token and not any(ch.isalpha() for ch in token):
                if tokens:
                    tokens[-1] = tokens[-1] + token
                else:
                    tokens.append(token)
            else:
                tokens.append(token)

        if len(tokens) != 28:
          return []

        lines = []

        for i in range(0, 28, 7):
          line = " ".join(tokens[i:i+7])
          lines.append(line)

        return [lines]

df['content'] = df['content'].apply(lambda x: split_content(x))

In [7]:
print(df['content'][0])

[]


In [8]:
df_exploded = df.explode('content')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=['content'])
df_exploded

,content,url
1,"[Nhất bộc rành rành lại thập thành,, Ở cho thự...",https://www.thivien.net/Tr%E1%BB%8Bnh-Doanh/Ba...
2,"[Văn chương phú lục đã xong rồi,, Thừa giấy là...",https://www.thivien.net/Nguy%E1%BB%85n-Qu%E1%B...
3,"[Đồn Phố Hiến vui hơn kinh kỳ,, Chơi ba ngày c...",https://www.thivien.net/Nguy%E1%BB%85n-Qu%E1%B...
5,"[Từ ghé non Bồng diễn bạn tiên,, Chạnh lòng kh...",https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi...
6,"[Thấy dân rét mướt, nghĩ mà thương,, Vậy phải ...",https://www.thivien.net/L%C3%AA-Th%C3%A1nh-T%C...
...,...,...
285,"[Từng đêm xuân cháy từng biển mộng, Mênh mông ...",https://www.thivien.net/H%C3%A0n-Qu%E1%BB%91c-...
286,"[Người ngồi như cỏ hoang vu quá!, Phố chết tưn...",https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...
287,"[Chỉ giống người trên mặt địa cầu, Đang tâm hà...",https://www.thivien.net/V%C5%A9-Ho%C3%A0ng-Ch%...
288,"[Hồ đã tan thây sóng Bạch Đằng, Minh còn mất v...",https://www.thivien.net/V%C5%A9-Ho%C3%A0ng-Ch%...


In [9]:
df_exploded['content'] = df_exploded['content'].apply(lambda x: '\n'.join(x))
df_exploded

,content,url
1,"Nhất bộc rành rành lại thập thành,\nỞ cho thực...",https://www.thivien.net/Tr%E1%BB%8Bnh-Doanh/Ba...
2,"Văn chương phú lục đã xong rồi,\nThừa giấy làm...",https://www.thivien.net/Nguy%E1%BB%85n-Qu%E1%B...
3,"Đồn Phố Hiến vui hơn kinh kỳ,\nChơi ba ngày ch...",https://www.thivien.net/Nguy%E1%BB%85n-Qu%E1%B...
5,"Từ ghé non Bồng diễn bạn tiên,\nChạnh lòng kha...",https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi...
6,"Thấy dân rét mướt, nghĩ mà thương,\nVậy phải l...",https://www.thivien.net/L%C3%AA-Th%C3%A1nh-T%C...
...,...,...
285,Từng đêm xuân cháy từng biển mộng\nMênh mông r...,https://www.thivien.net/H%C3%A0n-Qu%E1%BB%91c-...
286,Người ngồi như cỏ hoang vu quá!\nPhố chết tưng...,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...
287,Chỉ giống người trên mặt địa cầu\nĐang tâm hàn...,https://www.thivien.net/V%C5%A9-Ho%C3%A0ng-Ch%...
288,Hồ đã tan thây sóng Bạch Đằng\nMinh còn mất ví...,https://www.thivien.net/V%C5%A9-Ho%C3%A0ng-Ch%...


In [10]:
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset

Dataset({
    features: ['content', 'url', '__index_level_0__'],
    num_rows: 259
})

In [11]:
TEST_SIZE = 0.1
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'url', '__index_level_0__'],
        num_rows: 233
    })
    test: Dataset({
        features: ['content', 'url', '__index_level_0__'],
        num_rows: 26
    })
})

# Preprocess dataset

In [12]:
MODEL_NAME = 'danghuy1999/gpt2-viwiki'

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

In [14]:
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

def preprocess_function(row):
  return tokenizer(
      row['content'],
      padding='max_length',
      truncation=True,
      max_length=MAX_SEQ_LEN
  )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset['train'].column_names
)

Map (num_proc=4):   0%|          | 0/233 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/26 [00:00<?, ? examples/s]

In [21]:
tokenizer('xin chào các bạn', max_length=10, truncation=True, padding='max_length')

{'input_ids': [6504, 3939, 331, 1486, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

In [22]:
tokenized_poem_dataset['train'][0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [23]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Training

In [16]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [37]:
training_args = TrainingArguments(
    output_dir='poem_generation_gpt2',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    # fp16=true,
    push_to_hub=True
)

In [38]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset['train'],
    eval_dataset=tokenized_poem_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-38-4059e61a2066>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=300, training_loss=5.5461808268229165, metrics={'train_runtime': 299.2612, 'train_samples_per_second': 7.786, 'train_steps_per_second': 1.002, 'total_flos': 118908288000000.0, 'train_loss': 5.5461808268229165, 'epoch': 10.0})

# Inference

In [39]:
from transformers import pipeline

prompt = 'Con sông quê tôi đẹp\n'
generator = pipeline('text-generation', model='Dial2/poem_generation_gpt2')
results = generator(
    prompt,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = results[0]['generated_text']
print()
for line in results.split('\n'):
  print(line)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

Device set to use cuda:0



Con sông quê tôi đẹp
Làm tròn đôi ta cũng như anh. 
Năm nay, khi thấy chuyện người biết đâu?
Sống mãi cho biết ấy cũng chẳng nhớ.
Kêu khéo léo lại đã hết lòng!
Nước vẫn còn nhiều thế mà trời mưa.

